In [41]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [45]:
bike_trips = pd.read_csv('D:\\Capstone Data\\2014citibike\\2014-01 - Citi Bike trip data.csv', skiprows=1, names=['tripduration', 'starttime', 'stoptime', 'start_id',
       'start_name', 'start_latitude',
       'start_longitude', 'end_id', 'end_name',
       'end_latitude', 'end_longitude', 'bikeid', 'usertype',
       'birth_year', 'gender'])

In [47]:
bike_trips.head()

,tripduration,starttime,stoptime,start_id,start_name,start_latitude,start_longitude,end_id,end_name,end_latitude,end_longitude,bikeid,usertype,birth_year,gender
0,471,2014-01-01 00:00:06,2014-01-01 00:07:57,2009,Catherine St & Monroe St,40.711174,-73.996826,263,Elizabeth St & Hester St,40.717290,-73.996375,16379,Subscriber,1986,1
1,1494,2014-01-01 00:00:38,2014-01-01 00:25:32,536,1 Ave & E 30 St,40.741444,-73.975361,259,South St & Whitehall St,40.701221,-74.012342,15611,Subscriber,1963,1
2,464,2014-01-01 00:03:59,2014-01-01 00:11:43,228,E 48 St & 3 Ave,40.754601,-73.971879,2022,E 59 St & Sutton Pl,40.758491,-73.959206,16613,Subscriber,1991,1
3,373,2014-01-01 00:05:15,2014-01-01 00:11:28,519,Pershing Square N,40.751884,-73.977702,526,E 33 St & 5 Ave,40.747659,-73.984907,15938,Subscriber,1989,1
4,660,2014-01-01 00:05:18,2014-01-01 00:16:18,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,436,Hancock St & Bedford Ave,40.682166,-73.953990,19830,Subscriber,1990,1


In [48]:
for row in bike_trips.itertuples():
    print(row.start_id, row.start_latitude)
    break

2009 40.71117444


In [49]:
regions = pd.read_csv('D:\\Capstone Data\\taxizones.csv')

In [34]:

def extract_bike_stations(bike_trips, region_polygons, distinct_stations = {'station id':[], 'station name':[], 'station latitude':[], 'station longitude':[], 'station zone':[]}):
    '''
    Function to extract the distinct bike locations from citi bike trips data, and attribute each station a zone from the taxi zones

    Args:
        bike_trips (pd.DataFrame): DataFrame with bike trip data
        region_polygons (dict): dictionary with taxi zone id's as the keys and polygon objects as the values
        distinct_stations (dict): dictionary containing stations and their information, defaults to blank dictionary structure

    Returns:
        dict: A processed version of distinct_stations, with the addition of any newly encountered station id's
    '''

    for _, row in bike_trips.iterrows():
        if row.loc['start station id'] not in distinct_stations['station id']:
            distinct_stations['station id'].append(row.loc['start station id'])
            distinct_stations['station name'].append(row.loc['start station name'])
            distinct_stations['station latitude'].append(row.loc['start station latitude'])
            distinct_stations['station longitude'].append(row.loc['start station longitude'])

            s_point = Point(row.loc['start station longitude'], row.loc['start station latitude'])
            zone_set = False
            for zone, poly in region_polygons.items():
                if poly.contains(s_point):
                    distinct_stations['station zone'].append(zone)
                    zone_set = True
                    break
            if not zone_set:
                distinct_stations['station zone'].append(np.nan)
        if row.loc['end station id'] not in distinct_stations['station id']:
            distinct_stations['station id'].append(row.loc['end station id'])
            distinct_stations['station name'].append(row.loc['end station name'])
            distinct_stations['station latitude'].append(row.loc['end station latitude'])
            distinct_stations['station longitude'].append(row.loc['end station longitude'])

            e_point = Point(row.loc['end station longitude'], row.loc['end station latitude'])
            zone_set = False
            for zone, poly in region_polygons.items():
                if poly.contains(e_point):
                    distinct_stations['station zone'].append(zone)
                    zone_set = True
                    break
            if not zone_set:
                distinct_stations['station zone'].append(np.nan)
    
    return distinct_stations

In [35]:
region_polygons = {}

for _, row in regions.iterrows():
    poly = eval(row.loc['the_geom'])['coordinates'][0][0]
    region_polygons[row.loc['location_id']] = Polygon(poly)


distinct_stations = extract_bike_stations(bike_trips, region_polygons)

In [51]:
eval(regions.iloc[0].loc['the_geom'])['coordinates'][0][0]

[[-74.18445299999996, 40.694995999999904],
 [-74.18448899999999, 40.69509499999987],
 [-74.18449799999996, 40.69518499999987],
 [-74.18438099999997, 40.69587799999989],
 [-74.18428199999994, 40.6962109999999],
 [-74.18402099999997, 40.697074999999884],
 [-74.18391299999996, 40.69750699999986],
 [-74.18375099999997, 40.69779499999988],
 [-74.18363399999998, 40.6983259999999],
 [-74.18356199999994, 40.698451999999875],
 [-74.18354399999998, 40.69855999999988],
 [-74.18350799999996, 40.69870399999992],
 [-74.18327399999998, 40.70008999999988],
 [-74.18315699999994, 40.701214999999884],
 [-74.18316599999997, 40.702384999999886],
 [-74.18313899999998, 40.7026279999999],
 [-74.18309399999998, 40.7028529999999],
 [-74.18299499999995, 40.70315899999985],
 [-74.18284199999994, 40.70346499999989],
 [-74.18264399999998, 40.70373499999988],
 [-74.18242799999996, 40.70395099999992],
 [-74.18220299999996, 40.704139999999896],
 [-74.18203199999994, 40.70425699999987],
 [-74.18180699999994, 40.7043919

In [36]:
for (key, items) in distinct_stations.items():
    print(key, len(items))

station id 329
station name 329
station latitude 329
station longitude 329
station zone 329


In [37]:
bike_stations = pd.DataFrame(distinct_stations)
bike_stations.set_index('station id', inplace=True)

In [39]:
bike_stations[bike_stations['station zone']==np.nan]

,station name,station latitude,station longitude,station zone
station id,,,,


In [40]:
bike_stations.head()

,station name,station latitude,station longitude,station zone
station id,,,,
2009,Catherine St & Monroe St,40.711174,-73.996826,45.0
263,Elizabeth St & Hester St,40.717290,-73.996375,144.0
536,1 Ave & E 30 St,40.741444,-73.975361,137.0
259,South St & Whitehall St,40.701221,-74.012342,88.0
228,E 48 St & 3 Ave,40.754601,-73.971879,162.0
